# Crop Rotation (Rule-based)

Recommend next-season crops based on soil type, nutrient balance, and environment bands from `crop_soil.csv`.


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook')


In [4]:
# Load data and define rules
path_soil = os.path.join('..', 'datasets', 'crop_soil.csv')
soil = pd.read_csv(path_soil)
soil.columns = [c.strip().replace(' ', '_') for c in soil.columns]
soil['Soil_Type'] = soil['Soil_Type'].str.strip().str.title()
soil['Crop_Type'] = soil['Crop_Type'].str.strip().str.title()

In [5]:
suit_by_soil = soil.groupby('Soil_Type')['Crop_Type'].apply(lambda s: sorted(s.unique())).to_dict()

In [6]:
# Heuristic nutrient bias
soil['nutrient_bias'] = soil.apply(lambda r: 'N_high' if r['Nitrogen']>=r['Phosphorous'] and r['Nitrogen']>=r['Potassium'] else ('P_high' if r['Phosphorous']>=r['Nitrogen'] and r['Phosphorous']>=r['Potassium'] else 'K_high'), axis=1)
bias_by_crop = soil.groupby('Crop_Type')['nutrient_bias'].agg(lambda s: s.value_counts().idxmax()).to_dict()

bands = soil.groupby('Crop_Type').agg(
    temp_min=('Temparature', 'min'), temp_max=('Temparature', 'max'),
    moist_min=('Moisture', 'min'), moist_max=('Moisture', 'max'),
    hum_min=('Humidity', 'min'), hum_max=('Humidity', 'max')
).to_dict('index')

legumes = {'Pulses', 'Oil Seeds'}

In [9]:
def recommend_next_crop(current_crop, soil_type, temp, humidity, moisture, n, p, k, top_k=5):
    current_crop = str(current_crop).strip().title()
    soil_type = str(soil_type).strip().title()
    candidates = suit_by_soil.get(soil_type, [])
    scored = []
    cur_bias = bias_by_crop.get(current_crop)
    for cand in candidates:
        fam_penalty = 0.0 if cand != current_crop else -1.0
        cand_bias = bias_by_crop.get(cand)
        rot_bonus = 0.5 if cur_bias and cand_bias and cand_bias != cur_bias else 0.0
        if n < 15 and cand in legumes:
            rot_bonus += 0.3
        b = bands.get(cand)
        env_score = 0.0
        if b:
            if b['temp_min'] <= temp <= b['temp_max']:
                env_score += 0.3
            if b['moist_min'] <= moisture <= b['moist_max']:
                env_score += 0.3
            if b['hum_min'] <= humidity <= b['hum_max']:
                env_score += 0.2
        scored.append((cand, fam_penalty + rot_bonus + env_score))
    return sorted(scored, key=lambda x: x[1], reverse=True)[:top_k]

In [10]:
# Example
examples = [
    dict(current_crop='Paddy', soil_type='Clayey', temp=30, humidity=60, moisture=45, n=12, p=10, k=20),
    dict(current_crop='Wheat', soil_type='Loamy', temp=28, humidity=55, moisture=35, n=20, p=5, k=5),
]
for ex in examples:
    print('\nInput:', ex)
    print('Recommendations:', recommend_next_crop(**ex))


Input: {'current_crop': 'Paddy', 'soil_type': 'Clayey', 'temp': 30, 'humidity': 60, 'moisture': 45, 'n': 12, 'p': 10, 'k': 20}
Recommendations: [('Oil Seeds', 1.1), ('Pulses', 1.1), ('Barley', 0.8), ('Cotton', 0.8), ('Ground Nuts', 0.8)]

Input: {'current_crop': 'Wheat', 'soil_type': 'Loamy', 'temp': 28, 'humidity': 55, 'moisture': 35, 'n': 20, 'p': 5, 'k': 5}
Recommendations: [('Barley', 0.8), ('Cotton', 0.8), ('Ground Nuts', 0.8), ('Maize', 0.8), ('Millets', 0.8)]
